# Practical Session 5: Robotics challenge. Reactive navigation

## Laboratorio de Robótica 
### Grado en Ingeniería Electrónica, Mecatrónica y Robótica
### Universidad de Sevilla

## Objectives

In this practical session, we present a robotics challenge for the students, focusing on implementing autonomous navigation using the Turtlebot 3 robot. The ultimate objective is to build a ROS package capable of managing the robot's autonomous navigation in an environment with obstacles. This initial session will concentrate on reactive local navigation, where the robot must safely navigate to a given destination while avoiding surrounding obstacles. The task is divided into two parts: 1) controlling the robot to reach a designated waypoint, and 2) enabling the robot to follow a specified path to reach that waypoint.

We will first introduce some preliminary concepts that will aid in the project’s development before detailing the task at hand. Additionally, we provide supporting files that will be useful for developing and testing your system. In this session, you will learn more about:

+ Handling coordinate frames in ROS.
+ Managing ROS parameters using `rospy`.
+ Understanding how launch files work.
+ Completing and evaluating the challenge task: navigating the robot to a goal while avoiding obstacles.


## Coordinate frames 

ROS robots use multiple coordinate frames to operate. These are used to express coordinates of robot positions, waypoints, paths, and more. The different nodes in a ROS system need to know the existing frames and their relationships in order to transform coordinates properly. To handle this, there is a key package in ROS called `tf`. This package keeps track of multiple coordinate frames over time, maintaining the relationships between them in a tree structure buffered over time and published in the `/tf` topic. The `tf` package also provides functions to transform points, vectors, and other entities between any two coordinate frames at any desired point in time. It also includes tools to visualize the TF tree and other helpful features. For further information, check [here](http://wiki.ros.org/tf).

<figure style="text-align:center">
  <img src="images/turtlebot_tf.png" alt="" width=450>
  <figcaption>Fig. 1: TF tree of the Turtlebot 3.</figcaption>
</figure>

The main coordinate frames we will use with the Turtlebot robot are the following:

+ `/odom`. This is a static frame whose origin $(0,0,0)$ is set at the robot's initial position when it starts. This frame will be used as our _world_ coordinate system. 
+ `/base_footprint` (or `/base_link`). This is a local frame attached to the robot (that is, it moves with the robot), with the X-axis pointing in the direction of the robot's motion and the Z-axis pointing upwards.
+ `base_scan`. This frame is attached to the laser scanner mounted on the robot. 

<figure style="text-align:center">
  <img src="images/local_robot_tf.png" alt="" width=600>
  <figcaption>Fig. 2: Robot coordinate frames.</figcaption>
</figure>

Goal waypoints for the robot will typically be given in world coordinates, e.g., in the `/odom` frame. However, to control the robot’s motion, it may be useful to express the goal in the robot’s local coordinates, which allows us to reason about the linear ($v$) and angular ($\omega$) velocities needed to reach the destination. In Figure 2, the world frame `/odom` is denoted as $X_G$ and $Y_G$, while the robot’s local frame `/base_link` is denoted as $X_R$ and $Y_R$. The TF library can be used to transform points between coordinate frames, for example, using the `TransformListener` class. This class allows requests to transform data between any two given coordinate frames. The following code snippet, for instance, transforms a point from the `odom` frame to the `base_link` frame:

In [ ]:

import tf
from geometry_msgs.msg import PointStamped

goal = PointStamped();
base_goal = PointStamped();
goal.header.frame_id = "odom"

listener = tf.TransformListener()
listener.transformPoint('base_link', goal, base_goal);

## Parameters

In ROS, it's common to use parameters to configure nodes. These parameters are stored on the Parameter Server and can be retrieved by a node at runtime, for example, using the `rospy` library. The following data types are supported for parameter values:

+ 32-bit integers
+ Booleans
+ Strings
+ Doubles
+ ISO 8601 dates
+ Lists
+ Base64-encoded binary data

ROS parameters follow a hierarchical naming convention that aligns with the namespaces used for topics and nodes. This hierarchy helps prevent name collisions and allows parameters to be accessed either individually or as part of a tree structure.

### Private parameters
In the ROS naming convention, `~name` refers to a private parameter. Private parameters are primarily used for settings specific to a single node, intended for internal node configuration. For instance, in a node publishing laser scans, you might specify the range and resolution, while in a robot controller, you might configure the maximum allowable velocities for the vehicle.

### Reading parameters in a node

A parameter value stored on the Parameter Server can easily be retrieved within a ROS node (see examples [here](http://wiki.ros.org/rospy_tutorials/Tutorials/Parameters)). If the node is written in Python, you just need to call: `rospy.get_param(param_name)`:

In [ ]:
# get a global parameter
rospy.get_param('/global_param_name')
# get a parameter from our parent namespace
rospy.get_param('param_name')
# get a parameter from our private namespace
rospy.get_param('~private_param_name')

## Introduction to launch files

ROS provides `roslaunch`, a tool for easily launching multiple ROS nodes both locally and remotely (via SSH), as well as for setting parameters on the Parameter Server. It includes options to automatically respawn processes that have died. `roslaunch` uses one or more XML configuration files (with the `.launch` extension) to specify the parameters to set, the nodes to launch, and the machines that they should run on. Running a launch file will start all the nodes specified in the file with the provided parameters. You can start a launch file with the following command: `roslaunch package_name file_name.launch`.

The complete syntax for launch files can be checked [here](http://wiki.ros.org/roslaunch/XML). ROS nodes to be executed are included with the `<node>` tag. External arguments to a launch file can be defined using the `<arg>` tag. Other launch files can be included with the `<include>` tag, and their parameters can be set using additional `<arg>` tags. Using the `<param>` tag inside a node, you can define private parameters for that node: `<param name="param_name” type=""str|int|double|bool|yaml” value="value" />`. Alternatively, you can load multiple parameters from a YAML file with the following tag: `<rosparam command=”load” file=”PATH/TO/FILE.yaml”>`. 

The following example launch file would run a Turtlebot3 Gazebo simulation along with a node for teleoperating the robot. Another launch file is included to start the simulation, and a `<node>` tag is used to run a node of type `turtlebot3_teleop_key` (the executable node file) from the `turtlebot3_teleop` package.

In [ ]:
<launch>

<!-- find command is used to resolve the path of a package -->
<include file="$(find turtlebot3_gazebo)/launch/turtlebot3_empty_world.launch"/>

<node pkg="turtlebot3_teleop" type="turtlebot3_teleop_key" name="turtlebot3_teleop_keyboard"  output="screen">

<!-- we could place some param tags here -->

</node>

</launch>

## Challenge task: Reactive navigation avoiding obstacles

After covering some preliminary concepts essential for building our robotic system, it’s time to dive into the tasks for the robotics challenge. The ultimate objective is to develop a ROS package that enables autonomous navigation for the Turtlebot 3 robot. Our first task will be to implement a local reactive navigation system. We assume that our robot has differential drive and access to a localization system. The goal is to design a local controller that can safely navigate the robot to a specified goal, avoiding collisions with surrounding obstacles.

<figure style="text-align:center">
  <img src="images/local_nav_scheme.png" alt="" width=500>
  <figcaption>Fig. 3: Scheme of the local navigation system.</figcaption>
</figure>

Your system should provide velocity commands (both linear and angular) to the robot through the `/cmd_vel` topic, enabling it to navigate toward its goal. To ensure safe navigation, the robot must avoid obstacles, using its onboard 2D laser scanner for detection. This sensor provides readings consisting of a set of distances (ranges) to nearby obstacles, where each entry in the distance vector corresponds to a known angle, forming a set of polar coordinates relative to the sensor’s position. Your system should subscribe to laser scanner readings through the `/scan` topic, receiving messages of type `sensor_msgs::LaserScan`. For more details, see the message documentation [here](https://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/LaserScan.html). The `ranges[]` array provides obstacle distances at different angles, spanning from `angle_min` to `angle_max` in increments of `angle_increment`. 

The robot should operate in two navigation modes: 

+ Simple goal navigation: In this mode, the robot avoids obstacles as it navigates toward a single 2D waypoint goal. It should listen to the `move_base_simple/goal` topic, receiving messages of [type](http://docs.ros.org/en/noetic/api/geometry_msgs/html/msg/PoseStamped.html) `geometry_msgs/PoseStamped`. Each time a message arrives on this topic, the robot's goal will be updated. 

+ Path following: In this advanced mode, the robot navigates to its destination by following a given path, i.e., a sequence of discrete waypoints that lead to a final goal (see Figure 4 for an example). For this mode, the system should listen to the `path` topic, receiving messages of [type](https://docs.ros.org/en/noetic/api/nav_msgs/html/msg/Path.html) `nav_msgs/Path`.  

<figure style="text-align:center">
  <img src="images/path_example.png" alt="" width=650>
  <figcaption>Fig. 4: Example of a path (points) to a goal (red point).</figcaption>
</figure>

In the literature, there are numerous methods for reactive collision avoidance. You may design your local robot controller using any of these methods, create a variation of one, or develop your own algorithm to navigate the robot to a goal while avoiding obstacles. The supplementary materials for the robotics challenge provide additional information on the following algorithms:

+ Bug Algorithm
+ Potential Fields
+ Vector Field Histogram
+ Bubble Algorithm
+ Dynamic Window Approach
+ Velocity Obstacles

There are also several methods suitable for path following. In the supplementary material, you will find information about the next methods:

+ Carrot Chasing Algorithm (also known as the Follow-the-Carrot Algorithm)
+ Pure Pursuit Algorithm

__Note:__ The Pure Pursuit algorithm is generally designed for vehicles with steering control, where a desired curvature is specified. However, it can be adapted for differential drive vehicles (like the Turtlebot) by translating curvature into angular and linear velocities.

__Important:__ You are encouraged to develop your own solution for this challenge. Path following and collision avoidance may be implemented as separate ROS nodes or integrated into a single system. The algorithms provided are intended as starting points; you may implement or combine any of them or explore your own approach.


### Supporting files

In the Git repository of this course, within the `reactive_navigation` folder, we provide several files to help you develop and test your solution. 

+ launch:
  - `path_publisher.launch`: Launches a node for path publishing.
  - `turtlebot3_sim.launch`: Launches a simulation of the Turtlebot 3.
  - `challenge_reactive.launch`: Combines the above launch files to run the full challenge system. 

+ config:
  - `path.yaml`: YAML file containing coordinates for an example path.
  - `challenge_reactive.rviz`: RViz configuration file for loading a pre-configured visualization.

+ scripts:
  - `path_publisher.py`: Node that publishes a predefined path to the `path` topic, loaded from a YAML file.
  - `robot_controller.py`: Base node for implementing a robot controller for reactive collision avoidance. 

To test your solution, you can copy these files into your ROS package. Running the `challenge_reactive.launch` file will set up a complete test environment, including:

+ 1) a Gazebo simulation of the Turtlebot 3 in a scenario with obstacles,
+ 2) a `path_publisher.py` node that publishes a predefined path, and
+ 3) RViz with a pre-configured visualization.

You should then add your robot controller node for reactive local navigation. In order to write your node/s, you can use `robot_controller.py` as inspiration and complete it. 


### How to test your system

All goal waypoints and paths for the robot will be provided in a fixed frame, which we’ll designate as `odom` for this task. To test your controller’s ability to reach a goal waypoint, publish the goal to the `move_base_simple/goal` topic and observe the robot’s performance. You can set the goal using the following methods:

+ Command line: Use `rostopic pub` to publish directly to the `move_base_simple/goal` topic.
+ RViz: RViz provides an interface for setting navigation goals by clicking on a goal position. First, press the “2D Nav Goal” button (see Figure 5), then click on the desired location in the main visualization window. RViz will publish a `PoseStamped` message to the `move_base_simple/goal` topic, using the frame indicated in the “Fixed frame” field in the left menu, which should be `odom`.

<figure style="text-align:center">
  <img src="images/challenge_rviz.png" alt="" width=800>
  <figcaption>Fig. 5: RViz visualization for the challenge.</figcaption>
</figure>

To test your controller’s path-following functionality, publish a path to the `path` topic. The `path_publisher.launch` file will handle this automatically using the path specified in the `path.yaml` file.

__Exercise 1:__

Complete the robotics challenge by implementing a reactive collision avoidance controller that allows the Turtlebot 3 to navigate through goal waypoints and paths. Here’s what you need to do:

+ Create a new ROS package named `autonomous_navigation` in your workspace. Remember to update its dependencies as you develop the package further.
+ Copy the supporting files into the `scripts`, `launch`, and `config` folders.
+ Develop your package to complete the challenge task. You can create additional nodes, extend the provided launch files, or create new ones as needed.
+ Your controller and algorithms will have several paramenters (e.g., maximum robot velocities, lookahead distance, etc.). Consider loading them via the Parameter Server.
+ Update your Git repository with your completed code.
+ Complete your repository’s `README` file to explain: 1) code structure, that is, each node description and purpose; 2) node interfaces, that is, a list of data/topics each node subscribes to and publishes; and 3) instructions for launching and testing the system.

Note: A wise strategy might be to implement the task incrementally. First, you could devise a controller to reach a single goal, then extend it to handle collision avoidance, and finally, add path-following capabilities.  

Evaluation criteria: This task will be assessed based on several criteria, including the performance of your robot controller, the organization and structure of your code, the clarity and completeness of in-code documentation, and the quality of the repository’s `README` documentation.

### Performance evaluation

We provide a ROS package to evaluate the performance of each solution to the challenge. It includes a simulated scenario (Figure 5) where the robot is assigned a set of predefined goals. As the robot navigates toward these goals, a set of metrics will be computed to assess the controller's performance and compare it with other students' systems.

<figure style="text-align:center">
  <img src="images/evaluation_gazebo.png" alt="" width=600>
  <figcaption>Fig. 5: Simulated scenario for evaluation.</figcaption>
</figure>

In the course Git repository, you’ll find the `challenge_evaluation` package, which includes the following content:

+ `config`: RViz configuration files for visualizing the system (`robotics_challenge_reactive.rviz` and `robotics_challenge_planning.rviz`) and YAML file with predefined navigation goals (`navigation_goals.yaml`).
+ `launch`: Launch files to run the simulation along with the complete navigation system and the evaluation process. 
+ `scripts`: the `evaluation.py` node, which will be in charge of sending navigation goals to your system while computing performance metrics.

To evaluate the performance of your reactive navigation controller, start by running the simulation with the full system using the following command: `roslaunch challenge_evaluation robotics_challenge_reactive.launch`. Note that __you must complete the launch file__ to include the nodes from your `autonomous_navigation` package that implement the reactive controller. Once the simulation is running, launch the evaluation process with: `roslaunch challenge_evaluation evaluation_reactive.launch`. The `evaluation.py` node will then read the goals from the specified YAML file and send them sequentially. It will monitor whether each goal is reached __(a goal is considered reached when the robot is within 0.15 m)__ and will conclude after either reaching the final goal or a __200-second time limit__.

Upon completion, a file named `metrics_reactive.txt` will be created in the `config` folder, containing the following performance metrics:

+ __Goals reached:__ Boolean value that indicates whether all goals were reached (True) or not (False). 
+ __Elapsed time:__ Total time in seconds from the start to the end of the evaluation. 
+ __Traveled distance:__ Total distance (in meters) that the robot traveled from its starting position.
+ __Minimum distance to obstacles:__ Minimum distance (in meters) measured by the laser scanner to any obstacle.
+ __Average distance to obstacles:__ Average value of the minimum distances (in meters) to obstacles measured at each time step by the laser scanner.
+ __Collision penalties:__ A count of collisions with obstacles. A collision is considered each time the minimum distance detected by the laser scanner is 0.12 meters or less, adding 1 penalty per time step where this occurs.

__Exercise 2:__

Following the instructions above, evaluate the performance of your reactive navigation controller. Then, update your Git repository to include the resulting `metrics_reactive.txt` file. These metrics will be used to compare different solutions and assess your work.

## Summary

In this practical session, you should have learned the following:
+ How ROS handles TFs.
+ How to specify and read parameters for a ROS node.
+ How to use launch files to run multiple nodes. 
+ The steps required to implement the initial task in the robotics challenge: reactive navigation with collision avoidance.
+ How to assess the performance of your task solution. 
